In [1]:
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

# from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import panel as pn

pn.extension()

CHROMA_PATH = "input_data/chroma"
DATA_PATH = "input_data/"

/Users/divye/Desktop/meakuchatbot_project/meaku/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)

llm = ChatOllama(model="qwen:1.8b", temperature=0)

chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

question = "What are the advantages of using HackerEarth?"
result = chain.invoke({"query": question})

print(result["result"])

/Users/divye/Desktop/meakuchatbot_project/meaku/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/divye/Desktop/meakuchatbot_project/meaku/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/divye/Desktop/meakuchatbot_project/meaku/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It w

HackerEarth is an online technical-recruitment tool that provides a wide range of benefits to users who use it regularly.

1. Industry-Leading Features: HackerEarth offers industry-leading features that make it easy for organizations to recruit top talent quickly and efficiently. Some of the key features offered by HackerEarth include:

- Automated Recruitment Process: HackerEarth offers an automated recruitment process that helps organizations streamline their recruitment processes and find top talent more efficiently. This process uses a variety of tools and technologies, including artificial intelligence, machine learning, natural language processing (NLP), and other advanced technology solutions, to automate the recruitment process and help organizations find top talent more efficiently.
- Comprehensive Test Platform: HackerEarth offers a comprehensive test platform that enables organizations to create high-quality online technical assessments (TAs) that can help candidates prepare

In [11]:
import ollama


def generate_response(contents: str, user: str, chat_interface: pn.chat.ChatInterface):
    chat_history = chat_interface.serialize(
        format="transformers",
    )
    response = ollama.chat(model=chain, stream=True, messages=chat_history)
    message = ""
    for partial_resp in response:
        token = partial_resp["message"]["content"]
        message += token
        yield message


chat_interface = pn.chat.ChatInterface(callback=generate_response)

panel.chat.interface.ChatInterface

In [17]:
chat_interface.send(
    "Hi! How can i help you?", user="System", avatar="🤖", respond=False
)

In [14]:
chatbot = pn.Column(
    pn.pane.Markdown("# Qwen:1.8b Chatbot"),
    chat_interface,
    styles={
        "padding": "15px",
        "border": "1px solid white",
    },
)

In [16]:
chatbot.servable()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/divye/Desktop/meakuchatbot_project/meaku/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2024-08-11 23:43:30,501 Starting Bokeh server version 3.4.3 (running on Tornado 6.4.1)
2024-08-11 23:43:30,504 User authentication hooks NOT provided (default user enabled)
2024-08-11 23:43:30,506 Bokeh app running at: http://localhost:5006/
2024-08-11 23:43:30,506 Starting Bokeh server with process id: 24677
2024-08-11 23:43:31,815 WebSocket connection opened
2024-08-11 23:43:31,817 ServerConnection created
2024-08-11 23:43:42,047 WebSocket connection closed: code=1001, reason=None
2024-08-11 23:43:42,163 WebSocket connection opened
2024-08-11 23:43:42,164 ServerConnection created
^C

Interrupted, shutting down


In [2]:
llm = ChatOllama(model="qwen:1.8b", temperature=0)

RAG_PROMPT_TEMPLATE = """
Answer the question based on the following context:

{context}

---

Answer the question based on the above context: {question}
Do not repeat any information and be precise in your response.
"""

meaku_message = [
    (
        "system",
        "You are a helpful assistance for HackerEarth company. \
You first greet the customer in one line then collect their details such as name, email, contact number. \
You wait for the customer to respond then respond using the following context. \
{context_text} \
Answer the question using above context and donot repeat anything. \
You job is to engage customer asking about what are they looking for on the website so ask questions after responding. \
You respond in a short, very conversational friendly style. \
The company is about provide Human Resource services to their customers, the services include \
1. Repository of pre-built questions \
2. Assess a large pool of candidates in 38 different programming languages \
3. Versatile platform \
4. Advanced proctoring settings for assessments \
5. Detailed reports and analytics \
6. Dedicated account manager and product specialist available 24/7 \
7. Evaluates tests automatically to shortlist the best candidates \
If they are not looking for anything specific, you provide them details about what the company does and how its products can benefit the customer, \
             ",
    ),
    ("human", "{input_text}"),
]

prompt_template = ChatPromptTemplate.from_messages(meaku_message)


def get_context(query_text):
    embedding_function = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    return context_text


def get_completion_from_messages(prompt, input_text, model=llm):
    context_text = get_context(input_text)
    chain = prompt | model
    response_text = chain.invoke(
        {"context_text": context_text, "input_text": input_text}
    )
    # response = openai.ChatCompletion.create(
    #     model=model,
    #     messages=messages,
    #     temperature=0, # this is the degree of randomness of the model's output
    # )
    return response_text.content

## Manually testing the chat bot with three input texts

In [3]:
# messages = [
#     (
#         "system",
#         "You are a helpful assistant that translates English to German. Translate the user sentence.",
#     ),
#     ("human", "I love programming."),
# ]
# ai_msg = llm.invoke(messages)
# print(ai_msg.content)

# testing if context generating is working fine.

input_text = "What kind of services are offered by HackerEarth"
context_text = get_context(input_text)
# response = get_completion_from_messages(prompt, input_text, model=llm)
print(context_text)

/Users/divye/Desktop/MeakuChatBot/meaku/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/divye/Desktop/MeakuChatBot/meaku/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/divye/Desktop/MeakuChatBot/meaku/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by 

HackerEarth Assessment is designed to help you assess a large pool of candidates in a concise span of time. This means that you can quickly filter through a high volume of resumes and identify the best fit for your team. With HackerEarth Assessment, you can increase the number of candidates you assess, which can help you find the perfect candidate for your organization. This is particularly useful when you have multiple positions to fill or when you are looking to scale your team quickly. By automating the recruitment process, HackerEarth Assessment saves you time and effort and enables you to assess more candidates efficiently. This not only helps you find the right candidate faster but also ensures that you don't miss out on any potential hires.

3. Diverse platform

---

6. Dedicated account manager and product specialist available 24/7

HackerEarth Assessment offers exceptional service for its assessments, with a dedicated account manager and product specialist available 24/7 to as

In [4]:
input_text = "Hi, How are you?"
# prompt_template = ChatPromptTemplate.from_messages(meaku_message)
chain = prompt_template | llm
response = chain.invoke({"context_text": context_text, "input_text": input_text})
print(response.content)

Hello! I'm doing well, thank you for asking. Is there something specific you would like to know or discuss regarding HackerEarth Assessment or any of its services? I'd be happy to help answer your questions or provide additional information on the services offered by HackerEarth Assessment. Please let me know if you have any further questions or need more information on the services offered by HackerEarth Assessment.


In [5]:
input_text = "What kind of services are offered by HackerEarth"
chain = prompt_template | llm
response = chain.invoke({"context_text": context_text, "input_text": input_text})
print(response.content)

HackerEarth is a leading online technical recruitment platform that offers a wide range of services to its customers. Some of the key services offered by HackerEarth include:

  1. Pre-built questions repository: HackerEarth provides a vast repository of pre-built questions in various programming languages, including Python, Java, C++, and more. These pre-built questions are designed to help candidates quickly assess their skills and knowledge against a set of standardized test scenarios.
  2. Assess large pool of candidates in 38 different programming languages: HackerEarth provides an advanced platform for assessing large pools of candidates in 38 different programming languages, including Python, Java, C++, and more. This platform allows you to quickly assess the skills and knowledge of each candidate against a set of standardized test scenarios.
  3. Versatile platform: HackerEarth provides a versatile platform that enables you to quickly assess the skills and knowledge of each can

In [6]:
input_text = "Tell me more about pre-built questions repository."
chain = prompt_template | llm
response = chain.invoke({"context_text": context_text, "input_text": input_text})
print(response.content)

The pre-built questions repository on HackerEarth is a centralized platform that houses a vast collection of pre-built questions in various programming languages. The repository allows users to search for pre-built questions based on their specific requirements or interests.

Here are some key features and benefits of the pre-built questions repository on HackerEarth:

1. Comprehensive Collection: The repository contains a wide range of pre-built questions in various programming languages, including but not limited to Python, Java, C++, JavaScript, Ruby, Swift, and more. This comprehensive collection allows users to search for pre-built questions based on their specific requirements or interests.

2. Easy Search: The pre-built questions repository on HackerEarth provides an easy search feature that allows users to search for pre-built questions based on keywords or phrases entered by the user. This easy search feature ensures that users can quickly find and access the pre-built questio

In [7]:
panels = []  # collect display


def collect_messages(_):
    input_text = inp.value_input
    inp.value = ""
    prompt_template.extend([("human", f"{input_text}")])
    response = get_completion_from_messages(prompt_template, input_text, model=llm)
    prompt_template.extend([("system", f"{response}")])
    panels.extend(pn.Row("Human:", pn.pane.Markdown(input_text, width=600)))
    panels.extend(
        pn.Row(
            "System:",
            pn.pane.Markdown(
                response, width=600, styles={"background-color": "#F6F6F6"}
            ),
        )
    )

    return pn.Column(*panels)


inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here…")
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

/Users/divye/Desktop/MeakuChatBot/meaku/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BokehModel(combine_events=True, render_bundle={'docs_json': {'b17c0e44-6f82-480b-9063-2e696fc25eed': {'version…

/Users/divye/Desktop/MeakuChatBot/meaku/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/divye/Desktop/MeakuChatBot/meaku/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
